# Load data (from previous notebook)

In [10]:
import os
import urllib.request

# File name and URL
file_name = "sentences.txt"
url = "https://github.com/datanizing/m3-llm-workshop/raw/main/sentences.txt"

# Check if the file exists, if not, download it
if not os.path.isfile(file_name):
    print(f"{file_name} does not exist. Downloading...")
    urllib.request.urlretrieve(url, file_name)
    print(f"Downloaded {file_name}.")
else:
    print(f"{file_name} already exists.")

sentences.txt already exists.


In [11]:
sentences = open("sentences.txt", encoding="utf-8").read().split("@@@")

In [12]:
len(sentences)

20177

In [7]:
import numpy as np
with open("sentences-saev2.npy", "rb") as f:
    sembeddings = np.load(f)

In [8]:
with open("sentences-mpnet.npy", "rb") as f:
    sembeddings2 = np.load(f)

# Retrieval

In [5]:
import numpy as np
import pandas as pd
def search(query, text, corpus_embeddings, bi_encoder, cross_encoder, prompt_name="query", top_k=100):
    # code query to restrict search space
    query_embedding = bi_encoder.encode(query, prompt_name=prompt_name)

    # Determine similarity (vectors are normalized)
    sim = model.similarity(query_embedding, corpus_embeddings)[0].numpy()

    # Get most similar top_k by sorting
    hits = [ { "text": text[i], "score": sim[i] }
                     for i in sim.argsort()[::-1][0:top_k] ]

    # Consider only top hits for re-rankin
    cross_input = [[query, hit["text"]] for hit in hits]
    # cross-encode (this takes most time)
    cross_scores = cross_encoder.predict(cross_input)

    # Integrate cross-scores in original hits (this would be easier with pandas)
    for i in range(len(cross_scores)):
        hits[i]["cross-score"] = cross_scores[i]

    # nre-sort by cross-score, descending!
    hits = sorted(hits, key=lambda x: x["cross-score"], reverse=True)

    # Return top-20 results of re-ranker as dataframe
    return pd.DataFrame(hits[0:20])

In [1]:
# bi-encoder is needed
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("Snowflake/snowflake-arctic-embed-l-v2.0")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/203 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/250k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/784 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [2]:
model2 = SentenceTransformer('all-mpnet-base-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
# cross encoder
from sentence_transformers import CrossEncoder
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [6]:
pd.set_option('display.max_colwidth', 0)

In [13]:
search("Is the climate crisis worse for poorer countries?",
       sentences, sembeddings, model, cross_encoder, prompt_name="query")

,text,score,cross-score
0,"SIDs, though not the poorest countries, are extremely vulnerable to climate and other shocks, and lack resilience due to structural problems of limited human and financial resources, lack of economies of scale and higher costs due to their isolation from major manufacturing hubs.",0.504582,2.318397
1,"Climate change is causing geopolitical shifts in agriculture and fisheries, worsening the crises in countries that are vulnerable to food shortages.",0.481280,2.247720
2,"In that composite package of policies and measures, special consideration has to be accorded to the most vulnerable countries, such as small island developing States in the Caribbean and the Pacific and the poorer communities in climate-distressed areas of Africa.",0.495298,1.500093
3,"Our problems as poor countries tend to also become the problems of rich countries — demographic disparity, economic disparity, climate deregulation and the loss of institutional trust, among other things.",0.521800,1.290009
4,"As the global climate change crisis continues to worsen, so too will the issue of food insecurity and malnutrition.",0.466128,0.916754
5,More than half of the world’s top 50 most climate-vulnerable countries are home to 40 per cent of people living in extreme poverty.,0.590028,0.734999
6,"The cross-border financial impacts of crises, such as climate change and the pandemic, are impeding the ability of smaller indebted countries, such as mine, to make progress on the SDGs and climate adaptation and mitigation.",0.501148,0.654113
7,"As a result, countries like mine, which are unable to access concessionary funding, are forced to fix the climate crisis by obtaining loans at exorbitant rates from the very countries where the problem originated.",0.525503,0.466647
8,The climate crisis is another challenge that exacerbates the economic divide between nations and impedes humankind’s sustainable development.,0.518255,0.167014
9,"As with many conflicts, the main cause of climate crisis is a lack of trust and solidarity, coupled with the selfishness of some countries.",0.477392,0.124990


In [14]:
search("Is the climate crisis worse for poorer countries?",
       sentences, sembeddings2, model2, cross_encoder, prompt_name=None)

,text,score,cross-score
0,"Climate change is causing geopolitical shifts in agriculture and fisheries, worsening the crises in countries that are vulnerable to food shortages.",0.569836,2.247720
1,"Our problems as poor countries tend to also become the problems of rich countries — demographic disparity, economic disparity, climate deregulation and the loss of institutional trust, among other things.",0.569395,1.290009
2,More than half of the world’s top 50 most climate-vulnerable countries are home to 40 per cent of people living in extreme poverty.,0.660473,0.734999
3,"The cross-border financial impacts of crises, such as climate change and the pandemic, are impeding the ability of smaller indebted countries, such as mine, to make progress on the SDGs and climate adaptation and mitigation.",0.586076,0.654113
4,"As a result, countries like mine, which are unable to access concessionary funding, are forced to fix the climate crisis by obtaining loans at exorbitant rates from the very countries where the problem originated.",0.581408,0.466647
5,The climate crisis is another challenge that exacerbates the economic divide between nations and impedes humankind’s sustainable development.,0.659051,0.167014
6,"As with many conflicts, the main cause of climate crisis is a lack of trust and solidarity, coupled with the selfishness of some countries.",0.599951,0.124990
7,"The effects of climate change are causing suffering to the most vulnerable communities, especially small island developing States, least developed countries and those affected by conflict.",0.652519,0.073469
8,"We know that many States here, in particular the most vulnerable — those that have contributed the least to global warming and have burned the least fossil fuels — are the ones that have suffered the most from the climate crisis.",0.589185,-0.168129
9,No meaningful development can take place in countries that are also struggling with climate shocks and yet.,0.618825,-0.724854


In [15]:
search("Which countries are impacted most by the climate crisis?",
       sentences, sembeddings, model, cross_encoder)

,text,score,cross-score
0,Pakistan is one of the countries worst affected by the impacts of climate change.,0.531430,7.191168
1,"Developing countries, such as Cote d’Ivoire, which are only marginally responsible for climate change, are disproportionately affected and are suffering the most from its consequences.",0.583709,6.926192
2,The region that suffers the most from the impacts of climate change is the Caribbean and Central American region.,0.597967,6.228823
3,"Global crises such as pandemics, climate change, the war on Ukraine and the resulting geopolitical fragmentation have disproportionately impacted already fragile and conflict-affected countries.",0.555405,6.011687
4,Tajikistan is experiencing the impact of climate change first-hand and is considered one of the countries most vulnerable to the impact of climate change.,0.506089,5.855607
5,"We know that many States here, in particular the most vulnerable — those that have contributed the least to global warming and have burned the least fossil fuels — are the ones that have suffered the most from the climate crisis.",0.560039,5.842425
6,Central America and the Caribbean that are suffering the most devastating effects of climate change.,0.608699,5.308664
7,"Libya has been one of the most recent victims of the impacts of climate change, also affecting our country, which has experienced floods and record temperatures in recent months.",0.486841,5.247587
8,Libya and the other countries that have recently suffered the obvious ravages of climate change.,0.572106,5.189176
9,"As we mobilize to show up for Ukraine and countries that have experienced the devastating impact of climate shocks, including Libya.",0.497635,5.184596


In [16]:
search("Which countries are impacted most by the climate crisis?",
       sentences, sembeddings2, model2, cross_encoder, prompt_name=None)

,text,score,cross-score
0,Pakistan is one of the countries worst affected by the impacts of climate change.,0.635421,7.191168
1,"Developing countries, such as Cote d’Ivoire, which are only marginally responsible for climate change, are disproportionately affected and are suffering the most from its consequences.",0.652520,6.926192
2,The region that suffers the most from the impacts of climate change is the Caribbean and Central American region.,0.711692,6.228823
3,"Global crises such as pandemics, climate change, the war on Ukraine and the resulting geopolitical fragmentation have disproportionately impacted already fragile and conflict-affected countries.",0.617126,6.011687
4,Tajikistan is experiencing the impact of climate change first-hand and is considered one of the countries most vulnerable to the impact of climate change.,0.626007,5.855607
5,"We know that many States here, in particular the most vulnerable — those that have contributed the least to global warming and have burned the least fossil fuels — are the ones that have suffered the most from the climate crisis.",0.651732,5.842425
6,Central America and the Caribbean that are suffering the most devastating effects of climate change.,0.754463,5.308664
7,"Libya has been one of the most recent victims of the impacts of climate change, also affecting our country, which has experienced floods and record temperatures in recent months.",0.615565,5.247587
8,Libya and the other countries that have recently suffered the obvious ravages of climate change.,0.779879,5.189176
9,"Developing countries have made progress in reducing carbon emissions, but we continue to be the most affected by climate disasters.",0.652002,4.822717


In [ ]:
# eine von vielen weiteren Alternativen:
cross_encoder = CrossEncoder("jinaai/jina-reranker-v2-base-multilingual",
    automodel_args={"torch_dtype": "auto"},
    trust_remote_code=True)

In [20]:
search("Sind arme Länder stärker von der Klimakrise betroffen?",
       sentences, sembeddings, model, cross_encoder)

,text,score,cross-score
0,given that their impact includes third countries.,0.444092,-11.139576
1,This is no longer a crisis of the poorest or of the global South.,0.450066,-11.226036
2,But those that have done the least to cause the crisis in the first place are the ones most affected.,0.460511,-11.230425
3,"The rich countries have a responsibility, but ours cannot be excluded either.",0.475560,-11.322333
4,"and mainly for developing countries, where the gaps are much wider.",0.461101,-11.334226
5,"That is particularly the case in Ukraine and in the Sahel, the Great Lakes and Horn of Africa regions.",0.453107,-11.356651
6,"The devastating effect of climate change is contributing to increasing numbers of humanitarian crises, with record numbers of people in need of humanitarian assistance.",0.445204,-11.377127
7,"On the other hand, those countries are the least responsible for those crises.",0.497797,-11.378008
8,"The current global frameworks, shaped largely by the more affluent nations, remain largely unresponsive to those crises.",0.453424,-11.384809
9,And it is precisely the most vulnerable countries that are being hit the hardest.,0.531172,-11.396342
